In [1]:
import pandas as pd

In [2]:
# import zipcode to FIPS mapping
zip_mapping = pd.read_csv('ZIP-COUNTY-FIPS_2018-03.csv')
zip_mapping.rename(columns={"ZIP": "zipcode", "STCOUNTYFP": "fips", "CITY": "city_mapped", "STATE": "state_mapped", "COUNTYNAME":"county_mapped", "CLASSFP":"classfp"}, inplace = True)

In [3]:
appreciation = pd.read_csv('predicted_future_appreciation.csv')

In [4]:
zhvi = pd.read_csv('ZHVI/Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')
#zhvi = pd.melt(zhvi, id_vars=['RegionID','SizeRank','RegionName','RegionType','StateName','State','City','Metro','CountyName'])
zhvi.rename(columns={"RegionName": "zipcode", "StateName": "state", "City": "city", "CountyName": "county", "variable":"month", "2021-06-30":"zhvi"}, inplace = True)

zhvi = zhvi[["zipcode","state","city","county","zhvi"]]

In [5]:

rent = pd.read_csv('Zip_ZORI_AllHomesPlusMultifamily_SSA.csv')
rent.rename(columns={"RegionName": "zipcode", "2021-06": "rent"}, inplace = True)

rent = rent[["zipcode","rent"]]


In [6]:
zhvi.head()

,zipcode,state,city,county,zhvi
0,10025,NY,New York,New York County,1098866.0
1,60657,IL,Chicago,Cook County,516977.0
2,10023,NY,New York,New York County,1097064.0
3,77494,TX,Katy,Harris County,392858.0
4,60614,IL,Chicago,Cook County,658724.0


In [7]:
rent.zipcode.nunique()

1743

In [8]:
zhvi.zipcode.nunique()

30838

In [9]:
df = rent.merge(zhvi, how = "left", on = "zipcode")
df = df.merge(zip_mapping[["zipcode","fips"]], how = "left", on = "zipcode")
df = df.merge(appreciation[["zhvi_weighted","fips","population_growth","population","All industry total_growth","unemployment_rate","unemployment_rate_growth","predicted_3yr_appreciation"]], how = "left", on = "fips")



In [10]:


df["rent_vs_zhvi"] = df["rent"]/df["zhvi"]


In [11]:
df_cashflow = df[df.rent_vs_zhvi > 0.01]

#df[df.predicted_3yr_appreciation > 1.43]

In [12]:
df_cashflow.sort_values(by = "rent_vs_zhvi", ascending = False)

,zipcode,rent,state,city,county,zhvi,fips,zhvi_weighted,population_growth,population,All industry total_growth,unemployment_rate,unemployment_rate_growth,predicted_3yr_appreciation,rent_vs_zhvi
1888,21223,1137.0,MD,Baltimore,Baltimore City,58139.0,24510,188554.131693,0.963104,593490.0,1.110950,5.0,0.781250,1.160129,0.019557
663,63136,1006.0,MO,Jennings,Saint Louis County,59703.0,29189,225421.991870,0.996360,994205.0,1.098831,3.0,0.714286,1.186137,0.016850
664,63136,1006.0,MO,Jennings,Saint Louis County,59703.0,29510,164726.897767,0.961434,300576.0,1.102592,3.9,0.722222,1.190389,0.016850
692,48235,955.0,MI,Detroit,Wayne County,69260.0,26163,140081.128795,0.993599,1749343.0,1.088213,5.2,0.838710,1.290407,0.013789
1929,45405,825.0,OH,Dayton,Montgomery County,61497.0,39113,134602.875482,0.999754,531687.0,1.130659,4.2,0.840000,1.230172,0.013415
1914,40211,890.0,KY,Louisville,Jefferson County,67812.0,21111,187938.515964,0.999079,766757.0,1.069736,3.9,0.886364,1.190938,0.013125
1880,64130,915.0,MO,Kansas City,Jackson County,72887.0,29095,173804.462654,1.015281,703011.0,1.114871,3.7,0.725490,1.347990,0.012554
1055,38127,838.0,TN,Memphis,Shelby County,75258.0,47157,152025.859467,1.001223,937166.0,1.097521,4.0,0.740741,1.287902,0.011135
1559,48089,1068.0,MI,Warren,Macomb County,96405.0,26099,210581.053828,1.006261,873972.0,1.089814,4.3,0.826923,1.232669,0.011078
389,19134,1200.0,PA,Philadelphia,Philadelphia County,108591.0,42101,194443.288350,1.005084,1584064.0,1.066471,5.6,0.835821,1.343771,0.011051


In [19]:
df_filtered = df[(df.rent_vs_zhvi > 0.008) & (df.predicted_3yr_appreciation > 1.0)]
df_filtered.sort_values(by = "rent_vs_zhvi", ascending = False, inplace = True)
df_filtered.head(30)

/code/venvs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,zipcode,rent,state,city,county,zhvi,fips,zhvi_weighted,population_growth,population,All industry total_growth,unemployment_rate,unemployment_rate_growth,predicted_3yr_appreciation,rent_vs_zhvi
1888,21223,1137.0,MD,Baltimore,Baltimore City,58139.0,24510,188554.131693,0.963104,593490.0,1.110950,5.0,0.781250,1.160129,0.019557
664,63136,1006.0,MO,Jennings,Saint Louis County,59703.0,29510,164726.897767,0.961434,300576.0,1.102592,3.9,0.722222,1.190389,0.016850
663,63136,1006.0,MO,Jennings,Saint Louis County,59703.0,29189,225421.991870,0.996360,994205.0,1.098831,3.0,0.714286,1.186137,0.016850
692,48235,955.0,MI,Detroit,Wayne County,69260.0,26163,140081.128795,0.993599,1749343.0,1.088213,5.2,0.838710,1.290407,0.013789
1929,45405,825.0,OH,Dayton,Montgomery County,61497.0,39113,134602.875482,0.999754,531687.0,1.130659,4.2,0.840000,1.230172,0.013415
1914,40211,890.0,KY,Louisville,Jefferson County,67812.0,21111,187938.515964,0.999079,766757.0,1.069736,3.9,0.886364,1.190938,0.013125
1880,64130,915.0,MO,Kansas City,Jackson County,72887.0,29095,173804.462654,1.015281,703011.0,1.114871,3.7,0.725490,1.347990,0.012554
1055,38127,838.0,TN,Memphis,Shelby County,75258.0,47157,152025.859467,1.001223,937166.0,1.097521,4.0,0.740741,1.287902,0.011135
1559,48089,1068.0,MI,Warren,Macomb County,96405.0,26099,210581.053828,1.006261,873972.0,1.089814,4.3,0.826923,1.232669,0.011078
389,19134,1200.0,PA,Philadelphia,Philadelphia County,108591.0,42101,194443.288350,1.005084,1584064.0,1.066471,5.6,0.835821,1.343771,0.011051


In [15]:
df[df.zipcode == 30281]

,zipcode,rent,state,city,county,zhvi,fips,zhvi_weighted,population_growth,population,All industry total_growth,unemployment_rate,unemployment_rate_growth,predicted_3yr_appreciation,rent_vs_zhvi
114,30281,1618.0,GA,Stockbridge,Henry County,226866.0,13063,154370.299051,1.046264,292256.0,1.119448,4.4,0.656716,1.507349,0.007132
115,30281,1618.0,GA,Stockbridge,Henry County,226866.0,13151,173899.115213,1.061862,234561.0,1.239948,3.7,0.660714,1.432886,0.007132
116,30281,1618.0,GA,Stockbridge,Henry County,226866.0,13247,183305.189300,1.021453,90896.0,1.132375,3.9,0.661017,1.487828,0.007132
